In [42]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import layers
import os
import pandas as pd
import cv2

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [43]:
dataset_folder = "C:\\Users\\Aashish\\Desktop\\steeringmodel\\dataset"
final_images_folder_path = os.path.join(dataset_folder, 'final_images')
final_data_csv_path = os.path.join(dataset_folder, 'final_data.csv')

final_data = pd.read_csv(final_data_csv_path)

In [44]:
final_data

,timestamp,angle,throttle,brake
0,147942421588,0.001745,0.395651,0.148135
1,147942421598,0.003491,0.397726,0.148806
2,147942421608,0.006981,0.410178,0.147951
3,147942421618,0.026180,0.423972,0.148425
4,147942421628,0.031416,0.427024,0.148623
...,...,...,...,...
16103,147942655685,0.015708,0.308934,0.146639
16104,147942655695,0.019199,0.308934,0.147234
16105,147942655705,0.017453,0.308202,0.146929
16106,147942655715,0.013963,0.305272,0.146960


In [45]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16108 entries, 0 to 16107
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  16108 non-null  int64  
 1   angle      16108 non-null  float64
 2   throttle   16108 non-null  float64
 3   brake      16108 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 503.5 KB


In [46]:
final_data[final_data['timestamp'] == 147942421608]

,timestamp,angle,throttle,brake
2,147942421608,0.006981,0.410178,0.147951


In [47]:
import os
import cv2
import tensorflow as tf
import pandas as pd

class Data:
    def __init__(self, final_images_folder_path, final_data) -> None:
        self.path = final_images_folder_path
        self.final_data = final_data
        self.data = []

    def make_training_data(self):
        for filename in os.listdir(self.path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_name_as_ts = os.path.splitext(filename)[0]
                data_row = self.final_data[self.final_data['timestamp'] == int(image_name_as_ts)]
                if not data_row.empty:
                    angle = data_row['angle'].iloc[0]
                    throttle = data_row['throttle'].iloc[0]
                    brake = data_row['brake'].iloc[0]
                    self.data.append((filename, angle, throttle, brake))

    def load_image(self, filename):
        image = cv2.imread(os.path.join(self.path, filename))
        image = cv2.resize(image, (640, 480))  # Resize image to 320x240
        image = image / 255.0  # Normalize image data
        return image

    def data_generator(self, data):
        for filename, angle, throttle, brake in data:
            image = self.load_image(filename)
            yield image, (angle, throttle, brake)

    def create_dataset(self, data, batch_size, shuffle=True):
        dataset = tf.data.Dataset.from_generator(
            lambda: self.data_generator(data),
            output_signature=(
                tf.TensorSpec(shape=(480, 640, 3), dtype=tf.float32),  # Corrected shape
                (
                    tf.TensorSpec(shape=(), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.float32),
                )
            )
        )
        if shuffle:
            dataset = dataset.shuffle(buffer_size=len(data))
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        return dataset

    def training_data(self, batch_size):
        data_train = self.data[:int((len(self.data) * 3) / 4)]
        return self.create_dataset(data_train, batch_size)

    def validation_data(self, batch_size):
        data_val = self.data[int((len(self.data) * 3) / 4):]
        return self.create_dataset(data_val, batch_size, shuffle=False)

# Usage example:





# To use with TensorFlow model
# model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs)


In [48]:
# class Data():
#     def __init__(self) -> None:
#         self.path = final_images_folder_path
#         self.data = []
#         self.data_train = []
#         self.data_val = []
#         self.data_test = []
#     def make_training_data(self):
#         for filename in os.listdir(self.path):
#             if filename.endswith(".jpg") or filename.endswith(".png"):
#                 image = cv2.imread(os.path.join(self.path,filename))
#                 # image = cv2.resize(image, (320,120))
#                 image = image / 255.0  # Normalize image data
#                 image_name_as_ts = os.path.splitext(filename)[0]

#                 # print(f"img is {image}")

#                 data_row = final_data[final_data['timestamp'] == int(image_name_as_ts)]
#                 if not data_row.empty:
#                     angle = data_row['angle'].iloc[0]
#                     throttle = data_row['throttle'].iloc[0]
#                     brake = data_row['brake'].iloc[0]
#                     self.data.append((image, angle, throttle, brake))
        
#     def training_data(self):
#         self.data_train = self.data[:int((len(self.data)*3)/4)]
#         return self.data_train
    
#          # return last 1/4 of total data
#     def validation_data(self):
#         self.data_val = self.data[int((len(self.data)*3)/4)+1:-1]
#         return self.data_val
    
#     def testing_data(self):
#         self.data_test = self.data[int(len(self.data)-1):]
#         return self.data_test

In [49]:
# data = Data()
# data.make_training_data()

# data_train = data.training_data()
# print(type(data_train[0][0]))

In [50]:
# Input layer
inputs = keras.Input(name='input_shape', shape=(480, 640, 3))

# Convolutional feature maps
x = layers.Conv2D(filters=24, kernel_size=(5, 5), strides=(2, 2), activation='relu')(inputs)
x = layers.Conv2D(filters=36, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
x = layers.Conv2D(filters=48, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
x = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)
x = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)

# Flatten layer
x = layers.Flatten()(x)

# Fully connected layers with dropouts for overfit protection
x = layers.Dense(units=512, activation='relu')(x)
x = layers.Dropout(rate=0.1)(x)
x = layers.Dense(units=100, activation='relu')(x)
x = layers.Dropout(rate=0.1)(x)
x = layers.Dense(units=50, activation='relu')(x)
x = layers.Dropout(rate=0.1)(x)
x = layers.Dense(units=10, activation='relu')(x)
x = layers.Dropout(rate=0.1)(x)

# Derive steering angle value from single output layer by point multiplication
steering_angle = layers.Dense(units=1, activation='linear')(x)
steering_angle = layers.Lambda(lambda X: tf.multiply(tf.atan(X), 2), name='steering_angle')(steering_angle)

# Derive throttle pressure value from single output layer by point multiplication
throttle_press = layers.Dense(units=1, activation='linear')(x)
throttle_press = layers.Lambda(lambda X: tf.multiply(tf.atan(X), 2), name='throttle_press')(throttle_press)

# Derive brake pressure value from single output by point multiplication
brake_pressure = layers.Dense(units=1, activation='linear')(x)
brake_pressure = layers.Lambda(lambda X: tf.multiply(tf.atan(X), 2), name='brake_pressure')(brake_pressure)

# Build and compile model
model = keras.Model(inputs=[inputs], outputs=[steering_angle, throttle_press, brake_pressure])
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss={'steering_angle': 'mse', 'throttle_press': 'mse', 'brake_pressure': 'mse'}
)

# Model summary
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_shape (InputLayer)       [(None, 480, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_45 (Conv2D)             (None, 238, 318, 24  1824        ['input_shape[0][0]']            
                                )                                                                 
                                                                                                  
 conv2d_46 (Conv2D)             (None, 117, 157, 36  21636       ['conv2d_45[0][0]']              
                                )                                                           

In [51]:
x

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dropout_23')>

In [52]:
def train(model, name, data, epochs, steps, steps_val, batch_size):
    # Extract training and validation datasets
    train_dataset = data.training_data(batch_size)
    val_dataset = data.validation_data(batch_size)

    # Train the model using the datasets
    history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, steps_per_epoch=steps, validation_steps=steps_val)

    # Save the model
    model.save(f'{name}.h5')
    return history


In [53]:
# Assuming final_images_folder_path and final_data are already defined
data = Data(final_images_folder_path, final_data)
data.make_training_data()

batch_size = 16
epochs = 25
steps_per_epoch = 20
validation_steps = 20

# Build and compile the model as shown previously

# Train the model
train(model, 'jaishreeram', data, epochs, steps_per_epoch, validation_steps, batch_size)


Epoch 1/25
20/20 [==============================] - 90s 329ms/step - loss: 2.1948 - steering_angle_loss: 0.7605 - throttle_press_loss: 0.8195 - brake_pressure_loss: 0.6148 - val_loss: 0.1666 - val_steering_angle_loss: 0.1027 - val_throttle_press_loss: 0.0584 - val_brake_pressure_loss: 0.0054
Epoch 2/25
20/20 [==============================] - 5s 244ms/step - loss: 0.1292 - steering_angle_loss: 0.1087 - throttle_press_loss: 0.0155 - brake_pressure_loss: 0.0050 - val_loss: 0.1747 - val_steering_angle_loss: 0.1295 - val_throttle_press_loss: 0.0437 - val_brake_pressure_loss: 0.0015
Epoch 3/25
20/20 [==============================] - 5s 238ms/step - loss: 0.1136 - steering_angle_loss: 0.0957 - throttle_press_loss: 0.0140 - brake_pressure_loss: 0.0040 - val_loss: 0.1665 - val_steering_angle_loss: 0.1332 - val_throttle_press_loss: 0.0332 - val_brake_pressure_loss: 7.4305e-05
Epoch 4/25
20/20 [==============================] - 5s 240ms/step - loss: 0.0911 - steering_angle_loss: 0.0785 - thrott

In [55]:
import cv2
import numpy as np
from keras.models import load_model

def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Resize the image to match the expected input shape of the model
    image = cv2.resize(image, (640, 480))  # Correct dimensions
    # Normalize the pixel values to be in the range [0, 1]
    image = image.astype(np.float32) / 255.0
    # Expand dimensions to create a batch of one image
    image = np.expand_dims(image, axis=0)
    return image

def predict(image_paths, model_path):
    try:
        # Load the model
        model = load_model(model_path)
    except Exception as e:
        print("An unexpected error occurred:", e)
        print("Please check if the model file path is correct and the model file is not corrupted.")
        return None

    predictions = []
    for image_path in image_paths:
        # Preprocess the image
        preprocessed_image = preprocess_image(image_path)
        # Predict using the model
        prediction = model.predict(preprocessed_image)
        predictions.append(prediction)
    
    return predictions

# Example usage
test_image_paths = ["C:\\Users\\Aashish\\Desktop\\steeringmodel\\dataset\\center\\1479424259438336477.jpg"]
model_path = 'C:\\Users\\Aashish\\Desktop\\steeringmodel\\src\\jaishreeram.h5'
predictions = predict(test_image_paths, model_path)
if predictions is not None:
    print("Predictions:", predictions)


An unexpected error occurred: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]
Please check if the model file path is correct and the model file is not corrupted.


In [37]:
final_data[final_data['timestamp']==1479424201588]

,timestamp,angle,throttle,brake
